In [ ]:
# Chl_load_F1.4_python2

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import numpy as np
from netCDF4 import Dataset
from scipy.interpolate import griddata
from mpl_toolkits.basemap import Basemap

#from PiecewiseNorm import PiecewiseNorm
#import os.path
#import scipy.io as spio
#import pickle

modulepath   = '/home/ollie/mozeisin/pyfesom/'

import sys
sys.path.append(modulepath)

import pyfesom as pf1

# load FESOM 1.4 data

In [ ]:
resultpath = '/work/ollie/ncara/fesom/fesom-1.4-recom/v3/reccap_A_varCO2_varclim_1958_2019/'

#meshpath     = '/work/ollie/jhauck/input/meshes/CORE2/'
meshpath     = '/work/ollie/jhauck/input/meshes/core_new_288/'
#meshpath     = '/work/ollie/jhauck/input/meshes/core_new_384/'

# period of analysis 
first_year = 1960
last_year  = 1970
    
# number of upper ocean layers for surface mean
n_levels = 1

years = [first_year,last_year]
years_list      = np.arange(first_year,last_year+1,1)
months     = np.linspace(0,11,12).astype(int)

# dump data to this folder
dump = True
dumppath = '/work/ollie/mozeisin/evaluation/temp_data/FESOM1.4_'+str(first_year)+'_'+str(last_year)+'_'

In [ ]:
# load mesh --------------------------------------------------------------------------------------------
mesh = pf1.load_mesh(meshpath,abg=[0,0,0],get3d=True,usepickle=False)

In [ ]:
print(mesh.n2d)
#print(len(mesh.x2))

In [ ]:
# define FESOM output variables to use --------------------------------------------------------------------------------------------
runid='fesom'

var_PhyChl = 'bgc06' # [mg Chl m$^{-3}$]
var_PhyC = 'bgc05' # [mmol C m$^{-3}$]

var_DiaChl = 'bgc15' # [mg Chl m$^{-3}$]
var_DiaC = 'bgc14' # [mmol C m$^{-3}$]

# define storage arrays --------------------------------------------------------------------------------------------
PhyChl2D = np.zeros(len(mesh.x2))
PhyC2D = np.zeros(len(mesh.x2))
DiaChl2D = np.zeros(len(mesh.x2))
DiaC2D = np.zeros(len(mesh.x2))
#chl2D = np.zeros(len(mesh.x2))

#print(np.shape(PhyChl2D))

# load data --------------------------------------------------------------------------------------------
for ind in range(0,len(years_list)):
    print(years_list[ind])
    
    # Nanophyto Chl
    ncfilePhyChl = resultpath+var_PhyChl+'_'+runid+'_'+str(years_list[ind])+'0101.nc'
    f      = Dataset(ncfilePhyChl, 'r')
    phychl    = f.variables[var_PhyChl][:].mean(axis=0) # annual mean
    #print(np.shape(phychl[0:mesh.n2d]))
    PhyChl2D  = PhyChl2D + phychl[0,0:mesh.n2d] # select only surface layer
    #print(np.shape(PhyChl2D))

    if True: # set to False for test runs only! 
        # Nanophyto C
        ncfilePhyC = resultpath+var_PhyC+'_'+runid+'_'+str(years_list[ind])+'0101.nc'
        f      = Dataset(ncfilePhyC, 'r')
        phyc    = f.variables[var_PhyC][:].mean(axis=0)
        PhyC2D  = PhyC2D + phyc[0,0:mesh.n2d]

        # Diatom Chl
        ncfileDiaChl= resultpath+var_DiaChl+'_'+runid+'_'+str(years_list[ind])+'0101.nc'
        f     = Dataset(ncfileDiaChl, 'r')
        diachl    = f.variables[var_DiaChl][:].mean(axis=0)
        DiaChl2D  = DiaChl2D +diachl[0,0:mesh.n2d]

        # Diatom C
        ncfileDiaC= resultpath+var_DiaC+'_'+runid+'_'+str(years_list[ind])+'0101.nc'
        f     = Dataset(ncfileDiaC, 'r')
        diac    = f.variables[var_DiaC][:].mean(axis=0)
        DiaC2D  = DiaC2D +diac[0,0:mesh.n2d]

        # write sum of Chl storage arrays
        #chl2D  = chl2D + phy[0,0:mesh.n2d]+dia[0,0:mesh.n2d]
    
    
# data means --------------------------------------------------------------------------------------------    
PhyC2D_mean = PhyC2D/len(years_list)
PhyChl2D_mean = PhyChl2D/len(years_list)

DiaC2D_mean = DiaC2D/len(years_list)
DiaChl2D_mean = DiaChl2D/len(years_list)

#Chl2D_mean = chl2D/len(years_list)

#print(np.shape(PhyC2D_mean))


# Control: Plotting on original grid (basemap)

In [ ]:
elem2=mesh.elem[mesh.no_cyclic_elem,:]
d=PhyChl2D_mean[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k) 

figDChl = plt.figure(num=18, figsize=(8, 8), dpi=300, facecolor='w', edgecolor='k')
   
map = Basemap(projection='robin',lon_0=0)
x, y = map(mesh.x2, mesh.y2)
map.drawmapboundary(fill_color='0.9')
map.drawcoastlines()
mlabels=[False,False,False,False]
plabels=[True,True,True,True]
map.drawparallels(np.arange(-90,90,30),labels=plabels) #[1,0,0,0]
map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]

contoursDChl=np.array([0,0.02,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.18,0.2,0.22,0.24,0.26,0.28,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6])
eps=(contoursDChl.max()-contoursDChl.min())/50.
#data2[data2<=contours.min()]=contours.min()+eps
#data2[data2>=contours.max()]=contours.max()-eps

imDChl=plt.tricontourf(x, y, elem2, PhyChl2D_mean, levels=contoursDChl, cmap='viridis', 
                       #norm=PiecewiseNorm(contoursDChl), 
                       extend='max')

labelDChl = 'Nanophyto Chlorophyll a ('+str(years[0])+' to '+str(years[len(years)-1])+')' # in case of multiple years, replace ('+str(years[0])+') by ('+str(years[0])+' to '+str(years[len(years)-1])+')
plt.title(labelDChl)
cbarDChl=map.colorbar(imDChl,"bottom", size="5%", pad="2%")
cbarDChl.set_label(r'[mg Chl m$^{-3}$]') 

# Interpolation to regular grid

In [ ]:
lat         = np.arange(-89.5,90.,1.)
lon         = np.arange(-179.5,180.,1.)
latdic, londic = np.meshgrid(lat, lon)

lon_fesom = mesh.x2
lat_fesom = mesh.y2

print(np.shape(lon_fesom), np.shape(lat_fesom))

# Phyto Chl

In [ ]:
PhyChl2D_mean_interp = griddata( (lon_fesom, lat_fesom), PhyChl2D_mean.ravel(),
                              (londic, latdic), method='nearest')

PhyChl2D_mean_interp = np.ma.filled(PhyChl2D_mean_interp, np.nan)

print('\nFESOM PhyChl interpolation check [mg Chl/m3]:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
                np.nanmin(PhyChl2D_mean),np.nanmax(PhyChl2D_mean),np.nanmean(PhyChl2D_mean),
                np.nanmin(PhyChl2D_mean_interp),np.nanmax(PhyChl2D_mean_interp),np.nanmean(PhyChl2D_mean_interp)))
    

In [ ]:
### Interpolation check
figDChl = plt.figure(num=18, figsize=(8, 8), dpi=300, facecolor='w', edgecolor='k')
   
map = Basemap(projection='robin',lon_0=0)
x, y = map(londic, latdic)
map.drawmapboundary(fill_color='0.9')
map.drawcoastlines()
mlabels=[False,False,False,False]
plabels=[True,True,True,True]
map.drawparallels(np.arange(-90,90,30),labels=plabels) #[1,0,0,0]
map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]

contoursDChl=np.array([0,0.02,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.18,0.2,0.22,0.24,0.26,0.28,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6])
eps=(contoursDChl.max()-contoursDChl.min())/50.
#data2[data2<=contours.min()]=contours.min()+eps
#data2[data2>=contours.max()]=contours.max()-eps

imDChl=plt.pcolormesh(x, y, PhyChl2D_mean_interp, 
                      norm=colors.BoundaryNorm(boundaries=contoursDChl, ncolors=256), 
                      cmap='viridis', 
                       #norm=PiecewiseNorm(contoursDChl), 
                       )

labelDChl = 'Nanophyto Chlorophyll a ('+str(years[0])+' to '+str(years[len(years)-1])+')' # in case of multiple years, replace ('+str(years[0])+') by ('+str(years[0])+' to '+str(years[len(years)-1])+')
plt.title(labelDChl)
cbarDChl=map.colorbar(imDChl,"bottom", size="5%", pad="2%")
cbarDChl.set_label(r'[mg Chl m$^{-3}$]')

# Phyto C

In [ ]:
PhyC2D_mean_interp = griddata( (lon_fesom, lat_fesom), PhyC2D_mean.ravel(),
                              (londic, latdic), method='nearest')

PhyC2D_mean_interp = np.ma.filled(PhyC2D_mean_interp, np.nan)

print('\nFESOM PhyC interpolation check [mmol C/m3]:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
            np.nanmin(PhyC2D_mean),np.nanmax(PhyC2D_mean),np.nanmean(PhyC2D_mean),
            np.nanmin(PhyC2D_mean_interp),np.nanmax(PhyC2D_mean_interp),np.nanmean(PhyC2D_mean_interp)))
    

In [ ]:
### Interpolation check
figDChl = plt.figure(num=18, figsize=(8, 8), dpi=300, facecolor='w', edgecolor='k')
   
map = Basemap(projection='robin',lon_0=0)
x, y = map(londic, latdic)
map.drawmapboundary(fill_color='0.9')
map.drawcoastlines()
mlabels=[False,False,False,False]
plabels=[True,True,True,True]
map.drawparallels(np.arange(-90,90,30),labels=plabels) #[1,0,0,0]
map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]

#contoursDChl=np.array([0,0.02,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.18,0.2,0.22,0.24,0.26,0.28,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6])
eps=(contoursDChl.max()-contoursDChl.min())/50.
#data2[data2<=contours.min()]=contours.min()+eps
#data2[data2>=contours.max()]=contours.max()-eps

imDChl=plt.pcolormesh(x, y, PhyC2D_mean_interp, 
                      #norm=colors.BoundaryNorm(boundaries=contoursDChl, ncolors=256), 
                      cmap='viridis', 
                       #norm=PiecewiseNorm(contoursDChl), 
                       )

labelDChl = 'Nanophyto Carbon ('+str(years[0])+' to '+str(years[len(years)-1])+')' # in case of multiple years, replace ('+str(years[0])+') by ('+str(years[0])+' to '+str(years[len(years)-1])+')
plt.title(labelDChl)
cbarDChl=map.colorbar(imDChl,"bottom", size="5%", pad="2%")
cbarDChl.set_label(r'[mmol C m$^{-3}$]')

# Diatom Chl

In [ ]:
DiaChl2D_mean_interp = griddata( (lon_fesom, lat_fesom), DiaChl2D_mean.ravel(),
                              (londic, latdic), method='nearest')

DiaChl2D_mean_interp = np.ma.filled(DiaChl2D_mean_interp, np.nan)

print('\nFESOM DiaChl interpolation check [mg Chl/m3]:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
            np.nanmin(DiaChl2D_mean),np.nanmax(DiaChl2D_mean),np.nanmean(DiaChl2D_mean),
            np.nanmin(DiaChl2D_mean_interp),np.nanmax(DiaChl2D_mean_interp),np.nanmean(DiaChl2D_mean_interp)))
 

# Diatom C

In [ ]:
DiaC2D_mean_interp = griddata( (lon_fesom, lat_fesom), DiaC2D_mean.ravel(),
                              (londic, latdic), method='nearest')

DiaC2D_mean_interp = np.ma.filled(DiaC2D_mean_interp, np.nan)

print('\nFESOM DiaC interpolation check [mmol C/m3]:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
            np.nanmin(DiaC2D_mean),np.nanmax(DiaC2D_mean),np.nanmean(DiaC2D_mean),
            np.nanmin(DiaC2D_mean_interp),np.nanmax(DiaC2D_mean_interp),np.nanmean(DiaC2D_mean_interp)))
 

# export data to temp_data

In [ ]:
if dump:
    np.save(dumppath + 'PhyChl_mg_m3.npy', PhyChl2D_mean_interp, allow_pickle=False)
    np.save(dumppath + 'PhyC_mmol_m3.npy', PhyC2D_mean_interp, allow_pickle=False)
    
    np.save(dumppath + 'DiaChl_mg_m3.npy', DiaChl2D_mean_interp, allow_pickle=False)
    np.save(dumppath + 'DiaC_mmol_m3.npy', DiaC2D_mean_interp, allow_pickle=False)